In [ ]:
import pandas as pd

file_path2='/Users/gimhagyeong/test/B0005_with_lowess_features.csv'

df=pd.read_csv(file_path2)

In [ ]:
import numpy as np
import plotly.express as px
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler

# 1. 사용할 feature 선택
features1 = [
    'Voltage_measured', 'Current_measured', 'Temperature_measured',
    'Current_load', 'Voltage_load', 'Voltage_measured_smooth',
    'Voltage_measured_residual', 'Voltage_measured_trend',
    'Current_measured_smooth', 'Current_measured_residual',
    'Current_measured_trend', 'Temperature_measured_smooth',
    'Temperature_measured_residual', 'Temperature_measured_trend',
    'Current_load_smooth', 'Current_load_residual', 'Current_load_trend',
    'Voltage_load_smooth', 'Voltage_load_residual', 'Voltage_load_trend'
]

X = df[features1]

# 2. 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

lof = LocalOutlierFactor(
    n_neighbors=30,
    contamination=auto # 예상 이상치 비율
)

y_pred = lof.fit_predict(X_scaled)
lof_scores = -lof.negative_outlier_factor_

# threshold 설정 (직접 계산한 값 사용)
threshold = 1.676333654209548

df['lof_pred'] = y_pred                    # -1 / 1
df['lof_scores'] = lof_scores              # 실제 LOF score
df['lof_anomaly'] = (df['lof_scores'] > threshold).astype(int)  # 0/1로 변환

print("LOF 이상치:", df['lof_anomaly'].sum())


LOF 이상치: 638


기존 LOF baseline 모델(contamination=0.05)에서 결정된 threshold(1.6)를

고정된 기준으로 삼아,

LOWESS trend/residual 기반 features를 추가했을 때

동일 기준에서 anomaly detection sensitivity가 증가하는지 비교하였다.

결과적으로 638개의 anomaly window가 탐지되어, 파생변수 추가가 단발성 스파이크 및 국소 패턴 이상 탐지에 효과적임을 확인했다.

In [9]:
import plotly.express as px

cycle_df = df.groupby('cycle_idx')['lof_scores'].mean().reset_index()

fig = px.line(
    cycle_df,
    x='cycle_idx',
    y='lof_scores',      
    title='Average LOF Score per Cycle',
    markers=True
)
fig.show()

In [11]:
cycle_anom = df.groupby('cycle_idx')['lof_anomaly'].sum().reset_index()

fig = px.bar(
    cycle_anom,
    x='cycle_idx',
    y='lof_anomaly',
    title='Number of Anomalies per Cycle'
)
fig.show()


In [20]:
cycle_peak = df.groupby('cycle_idx')['lof_scores'].max().reset_index()
cycle_peak['is_anomaly'] = (cycle_peak['lof_scores'] > threshold).astype(int)

fig = px.line(
    cycle_peak,
    x='cycle_idx',
    y='lof_scores',
    title='Peak LOF Score per Cycle',
    markers=True
)
fig.show()

In [21]:
anom_cycles=df[df['lof_anomaly']==1]['cycle_idx'].unique()
print('Anomalic cycles:', anom_cycles)
print('number of Anomalic cycles:', len(anom_cycles))

Anomalic cycles: [  4   6   8  10  14  16  18  20  25  27  35  37  46  54  62  70  82  86
  90  94  98 102 106 110 114 118 122 130 134 136 144 148 150 154 158 166
 170 174 182 186 190 194 198 202 206 210 214 216 220 224 228 232 236 240
 244 248 252 256 260 264 270 274 278 286 290 294 298 302 306 316 320 324
 328 332 336 348 364 367 371 383 391 395 399 411 431 437 441 445 457 465
 469 473 477 481 483 485 493 509 517 525 529 533 537 541 545 548 556 564
 576 580 584 588 592 596 600 604 608 612 614]
number of Anomalic cycles: 119


In [27]:
cycle_anomaly_ratio = df.groupby('cycle_idx')['lof_anomaly'].mean().reset_index() # 이상 윈도우비율
# cycle_anomaly_ratio
cycle_anomaly_ratio['is_cycle_anomaly'] = (cycle_anomaly_ratio['lof_anomaly'] > 0.1).astype(int)
print("이상치윈도우가 0.1 초과인 싸이클 수:", cycle_anomaly_ratio['is_cycle_anomaly'].sum())
print("이상치윈도우가 0.1 초과인 싸이클 번호:", cycle_anomaly_ratio[cycle_anomaly_ratio['is_cycle_anomaly']==1]['cycle_idx'].values)

이상치윈도우가 0.1 초과인 싸이클 수: 4
이상치윈도우가 0.1 초과인 싸이클 번호: [134 150 154 614]


윈도우 기반 LOF는 단발성 이상이나 noise에 매우 민감하게 반응하여 fine-grain(아주 작고 세밀한 단위를 봄) 이상 패턴을 탐지한다.”

“그러나 cycle-level health anomaly를 판단하기 위해서는 이상 window 비율 기반의 2차 판단이 필요하다.”

“603개 window anomaly 중 cycle anomaly threshold(10%)를 통과한 cycle은 단 4개였다.”

“이는 변동성 신호는 곳곳에서 포착되지만 실제 열화(cycle-level structural degradation)는 일부 cycle에서만 나타난다는 것을 의미한다.